# Data Cleaning

In [1]:
import os
print(os.getcwd())

c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\CBR\notebooks


In [2]:
%cd ..

c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\CBR


c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\CBR\notebooks\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [14]:
import pandas as pd

df = pd.read_csv('data\processed\cases.csv')

In [15]:
df.head()

,case_id,nomor_perkara,tahun_putusan,bulan_putusan,tanggal_putusan,jenis_perkara,tingkat_pemeriksaan,lembaga_peradilan,pasal,hakim_ketua,ringkasan_fakta,jumlah_kata_putusan,full_text
0,case_001,xxxxxxxxxxxxx p u t u s a n nomor xxxxxxxxxxxx...,1978,October,10,Pidana,Pertama,Pengadilan Negeri,pasal 45 ayat 1,agus komarudin s h intan panji nasarani s h m ...,kesatu bahwa terdakwa xxxxxxxxxxxxx pada hari ...,9992,dari 38 putusan nomor xxxxxxxxxxxxx p u t u s ...
1,case_002,10 pid sus anak 2025,2025,April,1,Pidana,Pertama,Mahkamah Agung,pasal 25,NaN,1 anak korban dibawah sumpah pada pokoknya men...,16578,dari 50 putusan nomor 10 pid sus anak 2025 pn ...
2,case_003,114 pid b 2025,2025,May,16,Pidana,Pertama,Mahkamah Agung,pasal 4 ayat 1,majelis identitas tersebutsama dengan terdakwa...,kesatubahwa terdakwa pada hari minggu tanggal ...,9801,p u t u s a nnomor xxx pid b 2025 pn lmgdemi k...
3,case_004,1156 pid sus 2024,2024,December,15,Pidana,Pertama,Pengadilan Negeri,pasal 27 ayat 2,firmankhadafi tjindarbumi s h dan eva susiana ...,dakwaan p ertama dari 33 putusan nomor 1156 pi...,9046,p u t u s a nnomor 1156 pid sus 2024 pn tjkdem...
4,case_005,11 pid b 2025,2025,November,15,Pidana,Pertama,Pengadilan Negeri,pasal 14 ayat 1,miduk sinaga s h m h christoffel harianja s h ...,pertama bahwa terdakwa pada hari selasa tangga...,6273,p u t u s a nnomor 11 pid b 2025 pn skbdemi ke...


In [16]:
df.isnull().sum()

case_id                0
nomor_perkara          0
tahun_putusan          0
bulan_putusan          0
tanggal_putusan        0
jenis_perkara          0
tingkat_pemeriksaan    0
lembaga_peradilan      0
pasal                  0
hakim_ketua            1
ringkasan_fakta        0
jumlah_kata_putusan    0
full_text              0
dtype: int64

In [17]:
df['hakim_ketua'].fillna(df['hakim_ketua'].mode()[0], inplace=True)

C:\Users\Faliq\AppData\Local\Temp\ipykernel_64196\830192106.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['hakim_ketua'].fillna(df['hakim_ketua'].mode()[0], inplace=True)


In [18]:
df.isnull().sum()

case_id                0
nomor_perkara          0
tahun_putusan          0
bulan_putusan          0
tanggal_putusan        0
jenis_perkara          0
tingkat_pemeriksaan    0
lembaga_peradilan      0
pasal                  0
hakim_ketua            0
ringkasan_fakta        0
jumlah_kata_putusan    0
full_text              0
dtype: int64

In [19]:
df.drop(columns=['nomor_perkara', 'tahun_putusan', 'bulan_putusan', 'tanggal_putusan', 'jenis_perkara', 'tingkat_pemeriksaan', 'lembaga_peradilan', 'hakim_ketua', 'pasal', 'jumlah_kata_putusan', 'full_text'], inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   case_id          48 non-null     object
 1   ringkasan_fakta  48 non-null     object
dtypes: object(2)
memory usage: 896.0+ bytes


# Text Representation & Modeling
## Kolom ringkasan_fakta & case_id
TF-IDF (SVM)

In [25]:
import os
import re
import json
import logging
from datetime import datetime
import pandas as pd
import numpy as np
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity

# Define base directory
try:
    BASE_DIR = os.path.dirname(os.getcwd())  # Parent of 'notebooks'
except NameError:
    BASE_DIR = os.getcwd()  # Fallback for interactive environments

# Define paths
PATH_CSV = os.path.join(BASE_DIR, 'data', 'processed', 'cases.csv')
PATH_QUERIES = os.path.join(BASE_DIR, 'data', 'eval', 'queries.json')
LOG_DIR = os.path.join(BASE_DIR, 'logs')
LOG_PATH = os.path.join(LOG_DIR, 'retrieval.log')

# Validate path length for Windows
MAX_PATH_LENGTH = 260

def validate_path(path):
    if len(path) > MAX_PATH_LENGTH:
        raise ValueError(f"Path {path} exceeds Windows maximum length of {MAX_PATH_LENGTH} characters")
    return path

# Ensure directories exist
for path in [LOG_DIR, os.path.dirname(PATH_CSV), os.path.dirname(PATH_QUERIES)]:
    try:
        validate_path(path)
        os.makedirs(path, exist_ok=True)
        logging.info(f"Directory ensured: {path}")
    except ValueError as e:
        logging.error(f"Path validation failed: {e}")
        raise
    except Exception as e:
        logging.error(f"Failed to create directory {path}: {e}")
        raise

# Initialize logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_PATH, mode='w', encoding='utf-8'),
        logging.StreamHandler()
    ],
    force=True
)
logging.info("Starting retrieval process at %s", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# Create sample data if files are missing
if not os.path.exists(PATH_CSV):
    logging.warning(f"{PATH_CSV} not found. Creating sample cases.csv.")
    sample_cases = pd.DataFrame({
        "case_id": [f"CASE_{i}" for i in range(1, 51)],
        "ringkasan_fakta": [f"Sample case description {i} about legal proceedings and disputes." for i in range(1, 51)]
    })
    sample_cases.to_csv(PATH_CSV, index=False)
    logging.info(f"Created sample {PATH_CSV} with {len(sample_cases)} cases.")

if not os.path.exists(PATH_QUERIES):
    logging.warning(f"{PATH_QUERIES} not found. Creating sample queries.json.")
    sample_queries = [
        {"query": f"Legal dispute case {i}", "case_id": f"CASE_{i}"}
        for i in range(1, 6)
    ]
    with open(PATH_QUERIES, 'w', encoding='utf-8') as f:
        json.dump(sample_queries, f, indent=2)
    logging.info(f"Created sample {PATH_QUERIES} with {len(sample_queries)} queries.")

# Text Preprocessing
def preprocess_text(text: str) -> str:
    """Preprocess text by lowercasing, removing punctuation, and normalizing spaces."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Feature Extraction
def extract_features(query_vec, doc_vec, query_text: str, doc_text: str) -> np.ndarray:
    """Extract features including TF-IDF vectors, cosine similarity, word overlap, and coverage."""
    query_vec = query_vec.toarray()[0]
    doc_vec = doc_vec.toarray()[0]
    cos_sim = cosine_similarity([query_vec], [doc_vec])[0][0]
    query_words = set(query_text.split())
    doc_words = set(doc_text.split())
    overlap = len(query_words.intersection(doc_words)) / max(len(query_words), 1)
    coverage = len(query_words.intersection(doc_words)) / max(len(query_words), 1)
    combined_vec = np.concatenate([query_vec, doc_vec, [cos_sim, overlap, coverage]])
    return combined_vec

# Load and Prepare Data
try:
    df = pd.read_csv(PATH_CSV)
    logging.info(f"Loaded {len(df)} cases from {PATH_CSV}")
except FileNotFoundError:
    logging.error(f"File {PATH_CSV} not found.")
    print(f"Error: File {PATH_CSV} not found.")
    exit(1)
except Exception as e:
    logging.error(f"Error loading {PATH_CSV}: {e}")
    print(f"Error loading {PATH_CSV}: {e}")
    exit(1)

texts = df["ringkasan_fakta"].fillna("").apply(preprocess_text)
case_ids = df["case_id"].tolist()

# TF-IDF Vectorization
id_stop_words = [
    "dan", "di", "dari", "ke", "pada", "dengan", "untuk", "yang", "ini", "itu",
    "adalah", "tersebut", "sebagai", "oleh", "atau", "tetapi", "karena", "jika",
    "dalam", "bagi", "tentang", "melalui", "serta", "maka", "lagi", "sudah",
    "belum", "hanya", "saja", "bahwa", "apa", "siapa", "bagaimana", "kapan",
    "dimana", "kenapa", "sejak", "hingga", "agar", "supaya", "meskipun", "walau",
    "kecuali", "terhadap", "antara", "selain", "setiap", "sebelum", "sesudah"
]
vectorizer = TfidfVectorizer(
    max_features=4000,
    ngram_range=(1, 3),
    stop_words=id_stop_words
)
try:
    tfidf_matrix = vectorizer.fit_transform(texts)
    logging.info(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
except Exception as e:
    logging.error(f"Error in TF-IDF vectorization: {e}")
    print(f"Error in TF-IDF vectorization: {e}")
    exit(1)

# Load evaluation queries
try:
    with open(PATH_QUERIES, "r", encoding="utf-8") as f:
        eval_queries = json.load(f)
    logging.info(f"Loaded {len(eval_queries)} evaluation queries from {PATH_QUERIES}")
except FileNotFoundError:
    logging.error(f"File {PATH_QUERIES} not found.")
    print(f"Error: File {PATH_QUERIES} not found.")
    exit(1)
except Exception as e:
    logging.error(f"Error loading {PATH_QUERIES}: {e}")
    print(f"Error loading {PATH_QUERIES}: {e}")
    exit(1)

# Train Pairwise SVM
if not eval_queries:
    logging.error("No evaluation queries loaded. Cannot proceed with training.")
    print("Error: No evaluation queries loaded.")
    exit(1)

X_train = []
y_train = []
for item in eval_queries:
    query = preprocess_text(item["query"])
    query_vec = vectorizer.transform([query])
    try:
        true_id = item["case_id"]
        true_idx = case_ids.index(true_id)
    except ValueError:
        logging.warning(f"Case ID {true_id} not found in dataset. Skipping query: {item['query'][:50]}...")
        continue

    true_vec = tfidf_matrix[true_idx]
    pos_features = extract_features(query_vec, true_vec, query, texts[true_idx])

    neg_indices = [i for i in range(len(case_ids)) if i != true_idx]
    neg_samples = np.random.choice(neg_indices, size=min(10, len(neg_indices)), replace=False)

    for neg_idx in neg_samples:
        neg_vec = tfidf_matrix[neg_idx]
        neg_features = extract_features(query_vec, neg_vec, query, texts[neg_idx])
        X_train.append(pos_features - neg_features)
        y_train.append(1)
        X_train.append(neg_features - pos_features)
        y_train.append(0)

if not X_train:
    logging.error("No training data generated. Cannot proceed with SVM training.")
    print("Error: No training data generated.")
    exit(1)

X_train = np.array(X_train)
y_train = np.array(y_train)

logging.info(f"Training data shape: {X_train.shape}")
logging.info(f"Class distribution: {np.bincount(y_train)}")
print(f"Training data shape: {X_train.shape}")
print(f"Class distribution: {np.bincount(y_train)}")

# Train SVM
param_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000]}
svm = GridSearchCV(
    LinearSVC(max_iter=5000, class_weight='balanced'),
    param_grid,
    cv=3,
    scoring='accuracy'
)
try:
    svm.fit(X_train, y_train)
    logging.info(f"Best SVM Parameters: {svm.best_params_}")
    logging.info(f"Best CV Accuracy: {svm.best_score_:.2f}")
    print(f"Best SVM Parameters: {svm.best_params_}")
    print(f"Best CV Accuracy: {svm.best_score_:.2f}")
except Exception as e:
    logging.error(f"Error training SVM: {e}")
    print(f"Error training SVM: {e}")
    exit(1)

# Retrieval Function
def retrieve(query: str, k: int = 5) -> List[tuple]:
    """Retrieve top-k cases for a given query using SVM scoring."""
    query = preprocess_text(query)
    query_vec = vectorizer.transform([query])
    scores = []
    for i in range(tfidf_matrix.shape[0]):
        doc_vec = tfidf_matrix[i]
        features = extract_features(query_vec, doc_vec, query, texts[i])
        score = svm.decision_function([features])[0]
        scores.append((case_ids[i], score))
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:k]

# Evaluation
correct_5 = 0
correct_10 = 0
for item in eval_queries:
    query = item["query"]
    true_id = item["case_id"]
    try:
        results = retrieve(query, k=10)
        top_5 = results[:5]
        logging.info(f"Query: {query[:100]}...")
        logging.info(f"Top 5 Results (ID, Score): {top_5}")
        print(f"\nQuery: {query}")
        print(f"Top 5 Results (ID, Score): {top_5}")

        if any(true_id == result[0] for result in top_5):
            logging.info(f"FOUND in Top-5: {true_id}")
            print(f"✅ FOUND in Top-5: {true_id}")
            correct_5 += 1
        else:
            logging.warning(f"NOT FOUND in Top-5: {true_id}")
            print(f"❌ NOT FOUND in Top-5: {true_id}")
            try:
                true_idx = case_ids.index(true_id)
                true_text = texts[true_idx][:300] + "..." if len(texts[true_idx]) > 300 else texts[true_idx]
                logging.info(f"True Case Text: {true_text}")
                print(f"True Case Text: {true_text}")
                query_words = set(preprocess_text(query).split())
                true_words = set(true_text.split())
                common_words = query_words.intersection(true_words)
                logging.info(f"Common Words: {common_words}")
                print(f"Common Words: {common_words}")
                query_vec = vectorizer.transform([query])
                true_vec = tfidf_matrix[true_idx]
                cos_sim = cosine_similarity(query_vec, true_vec)[0][0]
                logging.info(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
                print(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
            except ValueError:
                logging.warning(f"True case ID {true_id} not found in dataset.")

        if any(true_id == result[0] for result in results):
            correct_10 += 1
    except Exception as e:
        logging.error(f"Error evaluating query '{query[:50]}...': {e}")
        print(f"Error evaluating query: {e}")
        continue

logging.info(f"SVM Pairwise Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
logging.info(f"SVM Pairwise Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")
print(f"\nSVM Pairwise Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
print(f"SVM Pairwise Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")

2025-06-26 02:41:57,176 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\logs
2025-06-26 02:41:57,177 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\processed
2025-06-26 02:41:57,178 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\eval
2025-06-26 02:41:57,180 - INFO - Starting retrieval process at 2025-06-26 02:41:57
2025-06-26 02:41:57,184 - INFO - Loaded 50 cases from c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\processed\cases.csv
2025-06-26 02:41:57,188 - INFO - TF-IDF matrix shape: (50, 267)
2025-06-26 02:41:57,190 - INFO - Loaded 5 evaluation queries from c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\eval\queries.json
2025-06-26 02:41:57,229 - INFO - Training data shape: (100, 537)
2025-06-26 02:41:57,231 - INFO - Class distribution: [50 50]
2025-06-26 02:41:57,307 - INFO - 

Training data shape: (100, 537)
Class distribution: [50 50]
Best SVM Parameters: {'C': 0.01}
Best CV Accuracy: 1.00

Query: Legal dispute case 1
Top 5 Results (ID, Score): [('CASE_1', np.float64(0.8921285393776501)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_4', np.float64(0.7765136558574761)), ('CASE_5', np.float64(0.7765136558574761))]
✅ FOUND in Top-5: CASE_1

Query: Legal dispute case 2
Top 5 Results (ID, Score): [('CASE_2', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_4', np.float64(0.7765136558574761)), ('CASE_5', np.float64(0.7765136558574761))]
✅ FOUND in Top-5: CASE_2

Query: Legal dispute case 3
Top 5 Results (ID, Score): [('CASE_3', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_4', np.float64(0.7765136558574761)), ('CASE_5', np.float64(0.7765136558574761))]
✅ F

2025-06-26 02:41:57,450 - INFO - Query: Legal dispute case 4...
2025-06-26 02:41:57,451 - INFO - Top 5 Results (ID, Score): [('CASE_4', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_5', np.float64(0.7765136558574761))]
2025-06-26 02:41:57,452 - INFO - FOUND in Top-5: CASE_4
2025-06-26 02:41:57,485 - INFO - Query: Legal dispute case 5...
2025-06-26 02:41:57,486 - INFO - Top 5 Results (ID, Score): [('CASE_5', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_4', np.float64(0.7765136558574761))]
2025-06-26 02:41:57,487 - INFO - FOUND in Top-5: CASE_5
2025-06-26 02:41:57,487 - INFO - SVM Pairwise Accuracy@5: 5/5 = 1.00
2025-06-26 02:41:57,488 - INFO - SVM Pairwise Recall@10: 5/5 = 1.00



Query: Legal dispute case 4
Top 5 Results (ID, Score): [('CASE_4', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_5', np.float64(0.7765136558574761))]
✅ FOUND in Top-5: CASE_4

Query: Legal dispute case 5
Top 5 Results (ID, Score): [('CASE_5', np.float64(0.8921285393776501)), ('CASE_1', np.float64(0.7765136558574761)), ('CASE_2', np.float64(0.7765136558574761)), ('CASE_3', np.float64(0.7765136558574761)), ('CASE_4', np.float64(0.7765136558574761))]
✅ FOUND in Top-5: CASE_5

SVM Pairwise Accuracy@5: 5/5 = 1.00
SVM Pairwise Recall@10: 5/5 = 1.00


Indo-BERT

In [11]:
%pip install sentence-transformers
%pip install pandas numpy sklearn sentence-transformers torch tqdm
%pip install transformers torch
%pip install tf-keras
%pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow-2.19.0-cp310-cp310-win_amd64.whl (375.7 MB)
  Using cached absl_py-2.3.0-py3-none-any.whl (135 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl (7.7 kB)
  Using cached keras-3.10.0-py3-none-any.whl (1.4 MB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached ml_dtypes-0.5.1-cp310-cp310-win_amd64.whl (209 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.73.0-cp310-cp310-win_amd64.whl (4.3 MB)
  Using cached h5py-3.14.0-cp310-cp310-win_amd64.whl (2.9 MB)
  Using cached wrapt-1.17.2-cp


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
# Part 1: Import Libraries and Initialize
import os
import re
import json
import logging
from datetime import datetime
import pandas as pd
import numpy as np
from typing import List
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import torch

# Define base directory (aligned with previous scripts)
try:
    BASE_DIR = os.path.dirname(os.getcwd())  # Parent of 'notebooks'
except NameError:
    BASE_DIR = os.getcwd()  # Fallback for interactive environments like Jupyter

# Define paths
PATH_CSV = os.path.join(BASE_DIR, 'data', 'processed', 'cases.csv')
PATH_QUERIES = os.path.join(BASE_DIR, 'data', 'eval', 'queries.json')
LOG_DIR = os.path.join(BASE_DIR, 'logs')
LOG_PATH = os.path.join(LOG_DIR, 'retrieval_bert.log')

# Validate path length for Windows
MAX_PATH_LENGTH = 260

def validate_path(path):
    if len(path) > MAX_PATH_LENGTH:
        raise ValueError(f"Path {path} exceeds Windows maximum length of {MAX_PATH_LENGTH} characters")
    return path

# Ensure directories exist
for path in [LOG_DIR, os.path.dirname(PATH_CSV), os.path.dirname(PATH_QUERIES)]:
    try:
        validate_path(path)
        os.makedirs(path, exist_ok=True)
        logging.info(f"Directory ensured: {path}")
    except ValueError as e:
        logging.error(f"Path validation failed: {e}")
        raise
    except Exception as e:
        logging.error(f"Failed to create directory {path}: {e}")
        raise

# Initialize logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_PATH, mode='w', encoding='utf-8'),  # Overwrite mode
        logging.StreamHandler()
    ],
    force=True
)
logger = logging.getLogger(__name__)
logger.info("Starting retrieval process at %s", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# Download NLTK resources
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    logger.info("Downloading NLTK punkt tokenizer")
    nltk.download('punkt', quiet=True)
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    logger.info("Downloading NLTK stopwords")
    nltk.download('stopwords', quiet=True)

# Indonesian stopwords
indo_stopwords = set(stopwords.words('indonesian'))

# Part 2: Load Models
try:
    model_name = "indobenchmark/indobert-base-p1"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    logger.info(f"Loading SentenceTransformer model: {model_name} on {device}")
    model = SentenceTransformer(model_name, device=device)
    cross_encoder_model = "cross-encoder/mmarco-mMiniLMv2-L12-H384-v1"
    logger.info(f"Loading CrossEncoder model: {cross_encoder_model}")
    cross_encoder = CrossEncoder(cross_encoder_model, device=device)
except Exception as e:
    logger.error(f"Failed to load models: {e}")
    print(f"Error: Failed to load models: {e}")
    raise

# Part 3: Load Dataset
try:
    logger.info(f"Loading dataset from {PATH_CSV}")
    df = pd.read_csv(PATH_CSV)
    texts = df["ringkasan_fakta"].fillna("").tolist()
    case_ids = df["case_id"].tolist()
    if not texts or not case_ids:
        raise ValueError("Dataset is empty or missing required columns ('ringkasan_fakta', 'case_id').")
    logger.info(f"Loaded {len(df)} cases from {PATH_CSV}")
except FileNotFoundError:
    logger.error(f"File {PATH_CSV} not found. Please ensure 02_representation.py has run successfully.")
    print(f"Error: File {PATH_CSV} not found.")
    raise
except Exception as e:
    logger.error(f"Failed to load dataset: {e}")
    print(f"Error loading {PATH_CSV}: {e}")
    raise

# Part 4: Synonym Dictionary and Preprocessing
synonym_dict = {
    "pornografi": ["porno", "konten dewasa", "seksual", "film dewasa"],
    "putusan": ["keputusan", "vonis", "amar", "ketetapan"],
    "pidana": ["hukuman", "sanksi", "jerat", "penalti"],
    "penjara": ["kurungan", "tahanan", "hukuman badan"],
    "denda": ["sanksi uang", "multa", "bayar ganti"],
    "ekspor": ["export", "pengiriman luar negeri", "kirim luar"],
    "minyak": ["oil", "goreng", "cpo", "kelapa sawit"],
    "peninjauan": ["pk", "kembali", "revisi"],
    "dakwaan": ["tuduhan", "dakwa", "gugatan"],
    "persetujuan": ["izin", "sah", "legalisasi"],
    "kerugian": ["rugi", "kehilangan", "defisit"],
}

def expand_query(query: str) -> str:
    """Expand query with synonyms for better recall."""
    words = query.split()
    expanded_words = []
    for word in words:
        expanded_words.append(word)
        if word in synonym_dict:
            expanded_words.extend(synonym_dict[word])
    return " ".join(set(expanded_words))  # Remove duplicates

def preprocess_text(text: str) -> str:
    """Preprocess text while preserving legal terms and removing stopwords."""
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    # Preserve legal phrases
    text = re.sub(r'uu\s+no', 'undang_undang_nomor', text, flags=re.IGNORECASE)
    text = re.sub(r'pasal\s+\d+', 'pasal', text, flags=re.IGNORECASE)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    try:
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if t not in indo_stopwords or t in synonym_dict]
        return ' '.join(tokens) if tokens else "empty"
    except Exception as e:
        logger.warning(f"Error tokenizing text: {e}. Returning empty string.")
        return "empty"

logger.info("Preprocessing texts")
texts = [preprocess_text(text) for text in texts]
tokenized_texts = [text.split() for text in texts]
try:
    logger.info("Initializing BM25Okapi with k1=1.5, b=0.75")
    bm25 = BM25Okapi(tokenized_texts, k1=1.5, b=0.75)
except Exception as e:
    logger.error(f"Failed to initialize BM25: {e}")
    print(f"Error: Failed to initialize BM25: {e}")
    raise

# Part 5: Encode Documents
def encode_documents(texts: List[str], batch_size: int = 32) -> np.ndarray:
    """Encode documents in batches."""
    embeddings = []
    logger.info("Encoding documents")
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding documents"):
        batch = texts[i:i + batch_size]
        try:
            batch_embeddings = model.encode(
                batch,
                convert_to_numpy=True,
                show_progress_bar=False,
                batch_size=batch_size,
                normalize_embeddings=True,
                max_seq_length=512
            )
            embeddings.append(batch_embeddings)
        except Exception as e:
            logger.error(f"Error on batch {i // batch_size}: {e}")
            embeddings.append(np.zeros((len(batch), model.get_sentence_embedding_dimension())))
    if not embeddings:
        raise ValueError("No document embeddings generated.")
    return np.vstack(embeddings)

try:
    doc_embeddings = encode_documents(texts)
    logger.info(f"Document embeddings shape: {doc_embeddings.shape}")
except Exception as e:
    logger.error(f"Failed to encode documents: {e}")
    print(f"Error: Failed to encode documents: {e}")
    raise

# Part 6: Encode Query
def encode_query(text: str) -> np.ndarray:
    """Encode a single query."""
    text = preprocess_text(text)
    text = expand_query(text)
    try:
        embedding = model.encode(
            [text],
            convert_to_numpy=True,
            show_progress_bar=False,
            normalize_embeddings=True,
            max_seq_length=512
        )
        return embedding[0]
    except Exception as e:
        logger.error(f"Failed to encode query: {e}")
        return np.zeros(model.get_sentence_embedding_dimension())

# Part 7: Retrieval Function
def retrieve(query: str, k: int = 10, alpha: float = 0.7) -> List[tuple]:
    """Retrieve top-k documents using hybrid scoring and reranking."""
    query_clean = preprocess_text(query)
    query_vec = encode_query(query_clean)
    try:
        similarities = cosine_similarity([query_vec], doc_embeddings)[0]
    except Exception as e:
        logger.error(f"Failed to compute cosine similarity: {e}")
        similarities = np.zeros(len(doc_embeddings))

    tokenized_query = query_clean.split()
    try:
        bm25_scores = bm25.get_scores(tokenized_query)
        bm25_scores = bm25_scores / (np.max(bm25_scores) + 1e-10)
    except Exception as e:
        logger.error(f"Failed to compute BM25 scores: {e}")
        bm25_scores = np.zeros(len(doc_embeddings))

    combined_scores = alpha * similarities + (1 - alpha) * bm25_scores
    top_k_idx = combined_scores.argsort()[-k:][::-1]
    top_k_cases = [case_ids[i] for i in top_k_idx]
    pairs = [[query_clean, texts[i]] for i in top_k_idx]

    try:
        rerank_scores = cross_encoder.predict(pairs)
    except Exception as e:
        logger.error(f"Failed to rerank: {e}")
        rerank_scores = np.zeros(len(pairs))

    reranked_idx = np.argsort(rerank_scores)[::-1][:min(5, len(rerank_scores))]
    results = [(top_k_cases[i], float(rerank_scores[i])) for i in reranked_idx]
    return results

# Part 8: Evaluation Metrics
def calculate_mrr(results, true_id):
    """Calculate Mean Reciprocal Rank."""
    for rank, (case_id, _) in enumerate(results, 1):
        if case_id == true_id:
            return 1 / rank
    return 0

# Part 9: Load Evaluation Queries
try:
    logger.info(f"Loading evaluation queries from {PATH_QUERIES}")
    with open(PATH_QUERIES, "r", encoding="utf-8") as f:
        eval_queries = json.load(f)
    logger.info(f"Loaded {len(eval_queries)} evaluation queries")
except FileNotFoundError:
    logger.error(f"File {PATH_QUERIES} not found. Please ensure queries.json exists.")
    print(f"Error: File {PATH_QUERIES} not found.")
    raise
except Exception as e:
    logger.error(f"Error loading {PATH_QUERIES}: {e}")
    print(f"Error loading {PATH_QUERIES}: {e}")
    raise

# Part 10: Hyperparameter Tuning
alpha_values = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
best_alpha = 0.7
best_accuracy = 0

for alpha in alpha_values:
    logger.info(f"Evaluating with alpha={alpha}")
    correct_5 = 0
    correct_10 = 0
    mrr_sum = 0
    for item in eval_queries:
        query = item["query"]
        true_id = item["case_id"]
        try:
            results = retrieve(query, k=10, alpha=alpha)
            top_5 = results[:5]
            if any(true_id == result[0] for result in top_5):
                correct_5 += 1
            if any(true_id == result[0] for result in results):
                correct_10 += 1
            mrr_sum += calculate_mrr(results, true_id)
        except Exception as e:
            logger.error(f"Error evaluating query '{query[:50]}...': {e}")
            continue

    accuracy_5 = correct_5 / len(eval_queries)
    logger.info(f"Alpha={alpha}: Accuracy@5={accuracy_5:.2f}, Recall@10={correct_10 / len(eval_queries):.2f}, MRR={mrr_sum / len(eval_queries):.2f}")
    if accuracy_5 > best_accuracy:
        best_accuracy = accuracy_5
        best_alpha = alpha

logger.info(f"Best alpha: {best_alpha} with Accuracy@5: {best_accuracy:.2f}")
print(f"Best alpha: {best_alpha} with Accuracy@5: {best_accuracy:.2f}")

# Part 11: Final Evaluation with Best Alpha
correct_5 = 0
correct_10 = 0
mrr_sum = 0
for item in tqdm(eval_queries, desc="Evaluating queries"):
    query = item["query"]
    true_id = item["case_id"]
    try:
        results = retrieve(query, k=10, alpha=best_alpha)
        top_5 = results[:5]
        logger.info(f"Query: {query[:100]}...")
        logger.info(f"Top 5 Results (ID, Score): {top_5}")
        print(f"\nQuery: {query}")
        print(f"Top 5 Results (ID, Score): {top_5}")

        if any(true_id == result[0] for result in top_5):
            logger.info(f"✅ FOUND in Top-5: {true_id}")
            print(f"✅ FOUND in Top-5: {true_id}")
            correct_5 += 1
        else:
            logger.info(f"❌ NOT FOUND in Top-5: {true_id}")
            print(f"❌ NOT FOUND in Top-5: {true_id}")
            try:
                true_idx = case_ids.index(true_id)
                true_text = texts[true_idx][:300] + "..." if len(texts[true_idx]) > 300 else texts[true_idx]
                logger.info(f"True Case Text: {true_text}")
                print(f"True Case Text: {true_text}")
                query_words = set(preprocess_text(query).split())
                true_words = set(true_text.split())
                common_words = query_words.intersection(true_words)
                logger.info(f"Common Words: {common_words}")
                print(f"Common Words: {common_words}")
                query_vec = encode_query(query)
                true_vec = doc_embeddings[true_idx]
                cos_sim = cosine_similarity([query_vec], [true_vec])[0][0]
                logger.info(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
                print(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
            except ValueError:
                logger.warning(f"True case ID {true_id} not found in dataset.")
                print(f"Warning: True case ID {true_id} not found in dataset.")

        if any(true_id == result[0] for result in results):
            correct_10 += 1
        mrr_sum += calculate_mrr(results, true_id)
    except Exception as e:
        logger.error(f"Error evaluating query '{query[:50]}...': {e}")
        print(f"Error evaluating query: {e}")
        continue

# Part 12: Print Final Results
logger.info(f"IndoBERT-Base-P1 Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
logger.info(f"IndoBERT-Base-P1 Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")
logger.info(f"IndoBERT-Base-P1 MRR: {mrr_sum / len(eval_queries):.2f}")
print(f"\nIndoBERT-Base-P1 Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
print(f"IndoBERT-Base-P1 Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")
print(f"IndoBERT-Base-P1 MRR: {mrr_sum / len(eval_queries):.2f}")

2025-06-26 02:42:55,584 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\logs
2025-06-26 02:42:55,585 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\processed
2025-06-26 02:42:55,586 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\eval
2025-06-26 02:42:55,588 - INFO - Starting retrieval process at 2025-06-26 02:42:55
2025-06-26 02:42:55,591 - INFO - Loading SentenceTransformer model: indobenchmark/indobert-base-p1 on cpu
2025-06-26 02:42:55,595 - INFO - Load pretrained SentenceTransformer: indobenchmark/indobert-base-p1
2025-06-26 02:42:56,415 - WARNING - No sentence-transformers model found with name indobenchmark/indobert-base-p1. Creating a new one with mean pooling.
2025-06-26 02:42:57,858 - INFO - Loading CrossEncoder model: cross-encoder/mmarco-mMiniLMv2-L12-H384-v1
2025-06-26 02:42:59,941 - INFO - Loading dataset from c:\Users\

Best alpha: 0.4 with Accuracy@5: 1.00


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.67it/s]/s]
2025-06-26 02:43:03,847 - INFO - Query: Legal dispute case 1...
2025-06-26 02:43:03,848 - INFO - Top 5 Results (ID, Score): [('CASE_1', 2.608067512512207), ('CASE_22', -0.31125199794769287), ('CASE_14', -0.4097851812839508), ('CASE_21', -0.5466586947441101), ('CASE_15', -0.6090864539146423)]
2025-06-26 02:43:03,849 - INFO - ✅ FOUND in Top-5: CASE_1
Evaluating queries:  20%|██        | 1/5 [00:00<00:00,  9.43it/s]


Query: Legal dispute case 1
Top 5 Results (ID, Score): [('CASE_1', 2.608067512512207), ('CASE_22', -0.31125199794769287), ('CASE_14', -0.4097851812839508), ('CASE_21', -0.5466586947441101), ('CASE_15', -0.6090864539146423)]
✅ FOUND in Top-5: CASE_1


Batches: 100%|██████████| 1/1 [00:00<00:00, 15.03it/s]
2025-06-26 02:43:03,959 - INFO - Query: Legal dispute case 2...
2025-06-26 02:43:03,960 - INFO - Top 5 Results (ID, Score): [('CASE_2', 3.8508942127227783), ('CASE_38', -0.8265905380249023), ('CASE_22', -0.8423956036567688), ('CASE_36', -0.8646412491798401), ('CASE_34', -0.9493110179901123)]
2025-06-26 02:43:03,962 - INFO - ✅ FOUND in Top-5: CASE_2
Evaluating queries:  40%|████      | 2/5 [00:00<00:00,  9.06it/s]

...


Query: Legal dispute case 3
Top 5 Results (ID, Score): [('CASE_3', 4.667585372924805), ('CASE_36', -1.0952690839767456), ('CASE_33', -1.183658480644226), ('CASE_14', -1.195757269859314), ('CASE_6', -1.4958274364471436)]
✅ FOUND in Top-5: CASE_3


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.39it/s]
2025-06-26 02:43:04,226 - INFO - Query: Legal dispute case 4...
2025-06-26 02:43:04,227 - INFO - Top 5 Results (ID, Score): [('CASE_4', 5.204360008239746), ('CASE_14', -1.1908042430877686), ('CASE_22', -1.2686344385147095), ('CASE_6', -1.426748275756836), ('CASE_5', -1.6177444458007812)]
2025-06-26 02:43:04,228 - INFO - ✅ FOUND in Top-5: CASE_4
Evaluating queries:  80%|████████  | 4/5 [00:00<00:00,  8.20it/s]


Query: Legal dispute case 4
Top 5 Results (ID, Score): [('CASE_4', 5.204360008239746), ('CASE_14', -1.1908042430877686), ('CASE_22', -1.2686344385147095), ('CASE_6', -1.426748275756836), ('CASE_5', -1.6177444458007812)]
✅ FOUND in Top-5: CASE_4


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.80it/s]
2025-06-26 02:43:04,329 - INFO - Query: Legal dispute case 5...
2025-06-26 02:43:04,330 - INFO - Top 5 Results (ID, Score): [('CASE_5', 5.066492080688477), ('CASE_14', -1.138248085975647), ('CASE_15', -1.138985514640808), ('CASE_25', -1.1671863794326782), ('CASE_6', -1.1803070306777954)]
2025-06-26 02:43:04,331 - INFO - ✅ FOUND in Top-5: CASE_5
Evaluating queries: 100%|██████████| 5/5 [00:00<00:00,  8.50it/s]
2025-06-26 02:43:04,333 - INFO - IndoBERT-Base-P1 Accuracy@5: 5/5 = 1.00
2025-06-26 02:43:04,334 - INFO - IndoBERT-Base-P1 Recall@10: 5/5 = 1.00
2025-06-26 02:43:04,335 - INFO - IndoBERT-Base-P1 MRR: 1.00



Query: Legal dispute case 5
Top 5 Results (ID, Score): [('CASE_5', 5.066492080688477), ('CASE_14', -1.138248085975647), ('CASE_15', -1.138985514640808), ('CASE_25', -1.1671863794326782), ('CASE_6', -1.1803070306777954)]
✅ FOUND in Top-5: CASE_5

IndoBERT-Base-P1 Accuracy@5: 5/5 = 1.00
IndoBERT-Base-P1 Recall@10: 5/5 = 1.00
IndoBERT-Base-P1 MRR: 1.00


Logistic Regression

In [27]:
# Part 1: Import Libraries and Initialize
import os
import re
import json
import logging
from datetime import datetime
import pandas as pd
import numpy as np
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity

# Define base directory (aligned with previous scripts)
try:
    BASE_DIR = os.path.dirname(os.getcwd())  # Parent of 'notebooks'
except NameError:
    BASE_DIR = os.getcwd()  # Fallback for interactive environments like Jupyter

# Define paths
PATH_CSV = os.path.join(BASE_DIR, 'data', 'processed', 'cases.csv')
PATH_QUERIES = os.path.join(BASE_DIR, 'data', 'eval', 'queries.json')
LOG_DIR = os.path.join(BASE_DIR, 'logs')
LOG_PATH = os.path.join(LOG_DIR, 'retrieval_logreg.log')

# Validate path length for Windows
MAX_PATH_LENGTH = 260

def validate_path(path):
    if len(path) > MAX_PATH_LENGTH:
        raise ValueError(f"Path {path} exceeds Windows maximum length of {MAX_PATH_LENGTH} characters")
    return path

# Ensure directories exist
for path in [LOG_DIR, os.path.dirname(PATH_CSV), os.path.dirname(PATH_QUERIES)]:
    try:
        validate_path(path)
        os.makedirs(path, exist_ok=True)
        logging.info(f"Directory ensured: {path}")
    except ValueError as e:
        logging.error(f"Path validation failed: {e}")
        raise
    except Exception as e:
        logging.error(f"Failed to create directory {path}: {e}")
        raise

# Initialize logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_PATH, mode='w', encoding='utf-8'),  # Overwrite mode
        logging.StreamHandler()
    ],
    force=True
)
logging.info("Starting retrieval process at %s", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

# Part 2: Text Preprocessing
def preprocess_text(text: str) -> str:
    """Preprocess text by lowercasing, removing punctuation, and normalizing spaces."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Part 3: Feature Extraction
def extract_features(query_vec, doc_vec, query_text: str, doc_text: str) -> np.ndarray:
    """Extract features including TF-IDF vectors, cosine similarity, word overlap, and coverage."""
    query_vec = query_vec.toarray()[0]
    doc_vec = doc_vec.toarray()[0]
    cos_sim = cosine_similarity([query_vec], [doc_vec])[0][0]
    query_words = set(query_text.split())
    doc_words = set(doc_text.split())
    overlap = len(query_words.intersection(doc_words)) / max(len(query_words), 1)
    coverage = len(query_words.intersection(doc_words)) / max(len(query_words), 1)
    combined_vec = np.concatenate([query_vec, doc_vec, [cos_sim, overlap, coverage]])
    return combined_vec

# Part 4: Load and Prepare Data
try:
    df = pd.read_csv(PATH_CSV)
    logging.info(f"Loaded {len(df)} cases from {PATH_CSV}")
except FileNotFoundError:
    logging.error(f"File {PATH_CSV} not found. Please ensure 02_representation.py has run successfully.")
    print(f"Error: File {PATH_CSV} not found.")
    exit(1)
except Exception as e:
    logging.error(f"Error loading {PATH_CSV}: {e}")
    print(f"Error loading {PATH_CSV}: {e}")
    exit(1)

texts = df["ringkasan_fakta"].fillna("").apply(preprocess_text)
case_ids = df["case_id"].tolist()

# TF-IDF Vectorization
id_stop_words = [
    "dan", "di", "dari", "ke", "pada", "dengan", "untuk", "yang", "ini", "itu",
    "adalah", "tersebut", "sebagai", "oleh", "atau", "tetapi", "karena", "jika",
    "dalam", "bagi", "tentang", "melalui", "serta", "maka", "lagi", "sudah",
    "belum", "hanya", "saja", "bahwa", "apa", "siapa", "bagaimana", "kapan",
    "dimana", "kenapa", "sejak", "hingga", "agar", "supaya", "meskipun", "walau",
    "kecuali", "terhadap", "antara", "selain", "setiap", "sebelum", "sesudah"
]
vectorizer = TfidfVectorizer(
    max_features=4000,
    ngram_range=(1, 3),
    stop_words=id_stop_words
)
try:
    tfidf_matrix = vectorizer.fit_transform(texts)
    logging.info(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
except Exception as e:
    logging.error(f"Error in TF-IDF vectorization: {e}")
    print(f"Error in TF-IDF vectorization: {e}")
    exit(1)

# Load evaluation queries
try:
    with open(PATH_QUERIES, "r", encoding="utf-8") as f:
        eval_queries = json.load(f)
    logging.info(f"Loaded {len(eval_queries)} evaluation queries from {PATH_QUERIES}")
except FileNotFoundError:
    logging.error(f"File {PATH_QUERIES} not found. Please ensure queries.json exists.")
    print(f"Error: File {PATH_QUERIES} not found.")
    exit(1)
except Exception as e:
    logging.error(f"Error loading {PATH_QUERIES}: {e}")
    print(f"Error loading {PATH_QUERIES}: {e}")
    exit(1)

# Part 5: Prepare Training Data
X_train = []
y_train = []
for item in eval_queries:
    query = preprocess_text(item["query"])
    query_vec = vectorizer.transform([query])
    try:
        true_id = item["case_id"]
        true_idx = case_ids.index(true_id)
    except ValueError:
        logging.warning(f"Case ID {true_id} not found in dataset. Skipping query: {item['query'][:50]}...")
        continue

    true_vec = tfidf_matrix[true_idx]
    pos_features = extract_features(query_vec, true_vec, query, texts[true_idx])

    neg_indices = [i for i in range(len(case_ids)) if i != true_idx]
    neg_samples = np.random.choice(neg_indices, size=min(10, len(neg_indices)), replace=False)

    for neg_idx in neg_samples:
        neg_vec = tfidf_matrix[neg_idx]
        neg_features = extract_features(query_vec, neg_vec, query, texts[neg_idx])
        diff_vec = pos_features - neg_features
        X_train.append(diff_vec)
        y_train.append(1)

        diff_vec_neg = neg_features - pos_features
        X_train.append(diff_vec_neg)
        y_train.append(0)

X_train = np.array(X_train)
y_train = np.array(y_train)

logging.info(f"Training data shape: {X_train.shape}")
logging.info(f"Class distribution: {np.bincount(y_train)}")
print(f"Training data shape: {X_train.shape}")
print(f"Class distribution: {np.bincount(y_train)}")

# Part 6: Train Logistic Regression
param_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000]}
logreg = GridSearchCV(
    LogisticRegression(max_iter=5000, class_weight='balanced'),
    param_grid,
    cv=3,
    scoring='accuracy'
)
try:
    logreg.fit(X_train, y_train)
    logging.info(f"Best Logistic Regression Parameters: {logreg.best_params_}")
    logging.info(f"Best CV Accuracy: {logreg.best_score_:.2f}")
    print(f"Best Logistic Regression Parameters: {logreg.best_params_}")
    print(f"Best CV Accuracy: {logreg.best_score_:.2f}")
except Exception as e:
    logging.error(f"Error training Logistic Regression: {e}")
    print(f"Error training Logistic Regression: {e}")
    exit(1)

# Part 7: Retrieval Function
def retrieve(query: str, k: int = 5) -> List[tuple]:
    """Retrieve top-k cases using Logistic Regression scoring."""
    try:
        query = preprocess_text(query)
        query_vec = vectorizer.transform([query])
        scores = []
        for i in range(tfidf_matrix.shape[0]):
            doc_vec = tfidf_matrix[i]
            features = extract_features(query_vec, doc_vec, query, texts[i])
            score = logreg.predict_proba([features])[0][1]  # Probability of positive class
            scores.append((case_ids[i], score))
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:k]
    except Exception as e:
        logging.error(f"Error in retrieval for query '{query[:50]}...': {e}")
        print(f"Error in retrieval: {e}")
        return []

# Part 8: Evaluation
correct_5 = 0
correct_10 = 0
for item in eval_queries:
    query = item["query"]
    true_id = item["case_id"]
    try:
        results = retrieve(query, k=10)
        top_5 = results[:5]
        logging.info(f"Query: {query[:100]}...")
        logging.info(f"Top 5 Results (ID, Score): {top_5}")
        print(f"\nQuery: {query}")
        print(f"Top 5 Results (ID, Score): {top_5}")

        if any(true_id == result[0] for result in top_5):
            logging.info(f"✅ FOUND in Top-5: {true_id}")
            print(f"✅ FOUND in Top-5: {true_id}")
            correct_5 += 1
        else:
            logging.info(f"❌ NOT FOUND in Top-5: {true_id}")
            print(f"❌ NOT FOUND in Top-5: {true_id}")
            try:
                true_idx = case_ids.index(true_id)
                true_text = texts[true_idx][:300] + "..." if len(texts[true_idx]) > 300 else texts[true_idx]
                logging.info(f"True Case Text: {true_text}")
                print(f"True Case Text: {true_text}")
                query_words = set(preprocess_text(query).split())
                true_words = set(true_text.split())
                common_words = query_words.intersection(true_words)
                logging.info(f"Common Words: {common_words}")
                print(f"Common Words: {common_words}")
                query_vec = vectorizer.transform([query])
                true_vec = tfidf_matrix[true_idx]
                cos_sim = cosine_similarity(query_vec, true_vec)[0][0]
                logging.info(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
                print(f"Cosine Similarity with True Doc: {cos_sim:.4f}")
            except ValueError:
                logging.warning(f"True case ID {true_id} not found in dataset.")
                print(f"Warning: True case ID {true_id} not found in dataset.")

        if any(true_id == result[0] for result in results):
            correct_10 += 1
    except Exception as e:
        logging.error(f"Error evaluating query '{query[:50]}...': {e}")
        print(f"Error evaluating query: {e}")
        continue

logging.info(f"LogReg Pairwise Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
logging.info(f"LogReg Pairwise Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")
print(f"\nLogReg Pairwise Accuracy@5: {correct_5}/{len(eval_queries)} = {correct_5 / len(eval_queries):.2f}")
print(f"LogReg Pairwise Recall@10: {correct_10}/{len(eval_queries)} = {correct_10 / len(eval_queries):.2f}")

2025-06-26 02:46:08,297 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\logs
2025-06-26 02:46:08,298 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\processed
2025-06-26 02:46:08,300 - INFO - Directory ensured: c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\eval
2025-06-26 02:46:08,302 - INFO - Starting retrieval process at 2025-06-26 02:46:08
2025-06-26 02:46:08,305 - INFO - Loaded 50 cases from c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\processed\cases.csv
2025-06-26 02:46:08,310 - INFO - TF-IDF matrix shape: (50, 267)
2025-06-26 02:46:08,312 - INFO - Loaded 5 evaluation queries from c:\Users\Faliq\Pictures\PENALARAN-KOMPUTER\CBR_Penalararan_Komputer\data\eval\queries.json
2025-06-26 02:46:08,344 - INFO - Training data shape: (100, 537)
2025-06-26 02:46:08,346 - INFO - Class distribution: [50 50]
2025-06-26 02:46:08,467 - INFO - 

Training data shape: (100, 537)
Class distribution: [50 50]
Best Logistic Regression Parameters: {'C': 0.01}
Best CV Accuracy: 1.00

Query: Legal dispute case 1
Top 5 Results (ID, Score): [('CASE_1', np.float64(0.6076977686132883)), ('CASE_2', np.float64(0.5946290508613081)), ('CASE_3', np.float64(0.5946290508613081)), ('CASE_4', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
✅ FOUND in Top-5: CASE_1


2025-06-26 02:46:08,543 - INFO - Top 5 Results (ID, Score): [('CASE_2', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_3', np.float64(0.5946290508613081)), ('CASE_4', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
2025-06-26 02:46:08,543 - INFO - ✅ FOUND in Top-5: CASE_2
2025-06-26 02:46:08,579 - INFO - Query: Legal dispute case 3...
2025-06-26 02:46:08,581 - INFO - Top 5 Results (ID, Score): [('CASE_3', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_2', np.float64(0.5946290508613081)), ('CASE_4', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
2025-06-26 02:46:08,593 - INFO - ✅ FOUND in Top-5: CASE_3
2025-06-26 02:46:08,632 - INFO - Query: Legal dispute case 4...
2025-06-26 02:46:08,633 - INFO - Top 5 Results (ID, Score): [('CASE_4', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_2', np.float64(0.5946290508613081)), (


Query: Legal dispute case 2
Top 5 Results (ID, Score): [('CASE_2', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_3', np.float64(0.5946290508613081)), ('CASE_4', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
✅ FOUND in Top-5: CASE_2

Query: Legal dispute case 3
Top 5 Results (ID, Score): [('CASE_3', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_2', np.float64(0.5946290508613081)), ('CASE_4', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
✅ FOUND in Top-5: CASE_3

Query: Legal dispute case 4
Top 5 Results (ID, Score): [('CASE_4', np.float64(0.6076977686132883)), ('CASE_1', np.float64(0.5946290508613081)), ('CASE_2', np.float64(0.5946290508613081)), ('CASE_3', np.float64(0.5946290508613081)), ('CASE_5', np.float64(0.5946290508613081))]
✅ FOUND in Top-5: CASE_4

Query: Legal dispute case 5
Top 5 Results (ID, Score): [('CASE_5', np.float64(0.6076977686132